<h2>Multilabel classification using CNN</h2>
By <a href="https://movsisyan.info/">Mher Movsisyan</a> and <a href="https://www.linkedin.com/in/tigran-avetisyan/">Tigran Avetisyan</a>

## 1 - Importing Tools and Data

In [1]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import Sequential
import matplotlib.patches as patches
from skimage.filters import threshold_otsu
from keras.optimizers import RMSprop, SGD, Adam
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from keract import get_activations, display_heatmaps, display_activations
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D, BatchNormalization, Reshape, GlobalAveragePooling2D, LeakyReLU
from keras.layers.experimental.preprocessing import RandomTranslation, RandomRotation, RandomWidth, RandomHeight, RandomZoom, Resizing

seed = 173
np.random.seed(seed)
tf.random.set_seed(seed)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18171858232440349427
]


In [2]:
df = pd.read_csv("../data/drawings_non_binary.csv")
mnist = pd.read_csv("../data/train.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5505,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


## 2 - Joining Data

In [3]:
joint = pd.DataFrame(mnist.iloc[:, 1:].to_numpy())
joint["label"] = mnist.label
jointDf = pd.DataFrame(np.concatenate((joint, df), axis=0)).rename({784: "label"}, axis=1).sample(frac=1, random_state=seed)

In [5]:
y = to_categorical(jointDf.label)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(jointDf.iloc[:,:-1], y, train_size=0.9, random_state=seed) # no need to stratify for large balanced datasets
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((42759, 784), (4751, 784), (42759, 10), (4751, 10))

In [11]:
model = keras.models.load_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
random_rotation (RandomRotat (None, 28, 28, 1)         0         
_________________________________________________________________
random_width (RandomWidth)   (None, 28, None, 1)       0         
_________________________________________________________________
random_translation (RandomTr (None, 28, None, 1)       0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 28, None, 1)       0         
_________________________________________________________________
resizing (Resizing)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        1

In [12]:
model.load_weights("../models/99.62/checkpoint.hdf5")

In [20]:
from keract import get_activations, display_heatmaps
activations = get_activations(model, np.array(x_train.iloc[4]/255).reshape(1, 1, 28*28))

In [21]:
activations.keys()

odict_keys(['reshape_input', 'reshape', 'random_rotation', 'random_width', 'random_translation', 'random_zoom', 'resizing', 'conv2d', 'batch_normalization', 'conv2d_1', 'batch_normalization_1', 'conv2d_2', 'batch_normalization_2', 'dropout', 'conv2d_3', 'batch_normalization_3', 'conv2d_4', 'batch_normalization_4', 'conv2d_5', 'batch_normalization_5', 'global_average_pooling2d', 'dropout_1', 'flatten', 'dense', 'leaky_re_lu', 'batch_normalization_6', 'dropout_2', 'dense_1'])

In [22]:
display_activations(activations = activations, save = True)

reshape_input (1, 1, 784) 
reshape (1, 28, 28, 1) 
random_rotation (1, 28, 28, 1) 
random_width (1, 28, 27, 1) 
random_translation (1, 28, 27, 1) 
random_zoom (1, 28, 27, 1) 
resizing (1, 28, 28, 1) 
conv2d (1, 28, 28, 64) 
batch_normalization (1, 28, 28, 64) 
conv2d_1 (1, 28, 28, 64) 
batch_normalization_1 (1, 28, 28, 64) 
conv2d_2 (1, 14, 14, 64) 
batch_normalization_2 (1, 14, 14, 64) 
dropout (1, 14, 14, 64) 
conv2d_3 (1, 14, 14, 128) 
batch_normalization_3 (1, 14, 14, 128) 
conv2d_4 (1, 14, 14, 128) 
batch_normalization_4 (1, 14, 14, 128) 
conv2d_5 (1, 7, 7, 128) 
batch_normalization_5 (1, 7, 7, 128) 
global_average_pooling2d (1, 128) 
dropout_1 (1, 128) 
flatten (1, 128) 
dense (1, 128) 
leaky_re_lu (1, 128) 
batch_normalization_6 (1, 128) 
dropout_2 (1, 128) 
dense_1 (1, 10) 
